In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import glob
import os
import numpy as np

# Full NCAA Dataset - 1:48 min
df = pd.read_csv("/content/drive/MyDrive/USC Baseball/Datasets/Full_22-24_NCAA_Dataset.csv")

<ipython-input-2-7315b4f966fa>:7: DtypeWarning: Columns (27,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/USC Baseball/Datasets/Full_22-24_NCAA_Dataset.csv")


In [16]:
returnerDF = df[(df['Date'] >= '2024-01-01') &
                 (df['Date'] <= '2024-12-30') &
                (df['TaggedPitchType'].isin(["Fastball", "Sinker", "FourSeamFastball"]))
                ]

# Returning the 'Name' column
# returnerNames = returnerDF['Pitcher'].unique()

In [3]:
returnerNames = ['Johnson, Fisher', 'Tostrup, Dylan', 'Ebner, Michael',
       'Matson, Sax', 'Di Landri, Tommy', 'Aoki, Caden', 'Blatter, Brock',
       'Austin, Channing', 'Edwards, Mason', 'Martinez, Xavier',
       'Blum, Josh', 'Stromsborg, Tyler', 'Hammond, Eric',
       'Purcell, Brodie', 'Ryhlick, Michael', 'Rizzo, Garren',
       'Feikes, Jared', 'Watson, Will', 'Harbour, Andrew', 'Baker, Jackson']

In [17]:
returnerDF['TaggedPitchType'].unique()

array(['Fastball', 'Sinker'], dtype=object)

In [18]:
topZoneDF = returnerDF[(returnerDF['PlateLocHeight'].between(3, 4)) &
                       (returnerDF['PlateLocSide'].between(-1, 1)) &
                       returnerDF['TaggedPitchType'].isin(['Fastball', 'Sinker', 'FourSeamFastball'])
                       ]

botZoneDF = returnerDF[(returnerDF['PlateLocHeight'].between(1, 2)) &
                       (returnerDF['PlateLocSide'].between(-1, 1)) &
                       returnerDF['TaggedPitchType'].isin(['Fastball', 'Sinker', 'FourSeamFastball'])
                       ]

In [19]:
for name in returnerNames:
  print(name)

Johnson, Fisher
Tostrup, Dylan
Ebner, Michael
Matson, Sax
Di Landri, Tommy
Aoki, Caden
Blatter, Brock
Austin, Channing
Edwards, Mason
Martinez, Xavier
Blum, Josh
Stromsborg, Tyler
Hammond, Eric
Purcell, Brodie
Ryhlick, Michael
Rizzo, Garren
Feikes, Jared
Watson, Will
Harbour, Andrew
Baker, Jackson


In [20]:
returning_pitcher_VAA = pd.DataFrame(columns=[
    'Name', 'Total FB', 'Release Height', 'Num Top FB', '% Top FB', 'Top Zone VAA', 'Top Zone Whiff%', 'Top Zone Chase%', 'Top Zone Hard Hit %', 'Top Zone BABIP', 'Top Zone SLG',
    'Num Bot FB', '% Bot FB', 'Bot Zone VAA', 'Bot Zone Whiff%', 'Bot Zone Chase%', 'Bot Zone Hard Hit %', 'Bot Zone BABIP', 'Bot Zone SLG', 'Top Location Rank', 'Bot Location Rank',
    'Whiff %', 'Chase %', 'Hard Hit %', 'BABIP', 'SLG'
])

def calculate_top_rank(avgTopVAA):
    if -3.7 <= avgTopVAA < 10:
        return "Elite High"
    elif -4.04 <= avgTopVAA < -3.71:
        return "Great High"
    elif -4.42 <= avgTopVAA < -4.04:
        return "Above Average High"
    elif -4.93 <= avgTopVAA < -4.42:
        return "Average High"
    elif -5.31 <= avgTopVAA < -4.93:
        return "Below Average High"
    elif -5.69 <= avgTopVAA < -5.31:
        return "Poor High"
    elif -99 <= avgTopVAA < -5.69:  # Adjusted range for Awful High
        return "Awful High"

def calculate_bot_rank(avgBotVAA):
    if -5.54 <= avgBotVAA < 100:
        return "Elite Low"
    elif -5.96 <= avgBotVAA < -5.54:
        return "Great Low"
    elif -6.35 <= avgBotVAA < -5.96:
        return "Above Average Low"
    elif -6.89 <= avgBotVAA < -6.35:
        return "Average Low"
    elif -7.3 <= avgBotVAA < -6.89:
        return "Below Average Low"
    elif -7.71 <= avgBotVAA < -7.3:
        return "Poor Low"
    elif -100 <= avgBotVAA < -7.71:  # Adjusted range for Awful Low
        return "Awful Low"

def calculate_stats(topZoneDF, botZoneDF, returnerDF, playerName):
    stats = {}
    filteredTop_df = topZoneDF[topZoneDF['Pitcher'] == playerName]
    filteredBot_df = botZoneDF[botZoneDF['Pitcher'] == playerName]
    filtered_df = returnerDF[returnerDF['Pitcher'] == playerName]

    if filteredTop_df.empty and filteredBot_df.empty:
        return None

    out_zone = filtered_df[~((filtered_df['PlateLocSide'].between(-0.951, 0.951)) & (filtered_df['PlateLocHeight'].between(1.5, 3.6)))]
    o_pitch = len(out_zone)
    o_swing = out_zone['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall']).sum()
    stats['Chase %'] = o_swing / o_pitch if o_pitch > 0 else 0

    strike_swing = filtered_df['PitchCall'].isin(['StrikeSwinging'])
    swing = filtered_df['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall'])
    stats['Whiff %'] = strike_swing.sum() / swing.sum() if swing.sum() > 0 else 0

    stats['Hard Hit %'] = (
        (filtered_df['PitchCall'].isin(['InPlay']) &
        (filtered_df['ExitSpeed'] >= 95)).sum() /
        (filtered_df['PitchCall'].isin(['InPlay']) &
        filtered_df['ExitSpeed'].notna()).sum()
    )

    stats['BABIP'] = (filtered_df[filtered_df['PlayResult'].isin(['Single', 'Double', 'Triple', 'HomeRun'])]).shape[0] / filtered_df[filtered_df['PitchCall'] == "InPlay"].shape[0]
    stats['SLG'] = (
        filtered_df[filtered_df['PlayResult'] == "Single"].shape[0] +
        2 * filtered_df[filtered_df['PlayResult'] == "Double"].shape[0] +
        3 * filtered_df[filtered_df['PlayResult'] == "Triple"].shape[0] +
        4 * filtered_df[filtered_df['PlayResult'] == "HomeRun"].shape[0]
    ) / filtered_df[filtered_df['PitchCall'] == "InPlay"].shape[0]

    stats['Total FB'] = len(filtered_df)
    stats['Release Height'] = filtered_df['RelHeight'].mean()
    stats['Num Top FB'] = len(filteredTop_df)
    stats['% Top FB'] = stats['Num Top FB'] / stats['Total FB']
    stats['Top Zone VAA'] = filteredTop_df['VertApprAngle'].mean()
    stats['Top Zone Whiff%'] = filteredTop_df['PitchCall'].isin(['StrikeSwinging']).sum() / filteredTop_df['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall']).sum()
    stats['Top Zone Hard Hit %'] = (
        (filteredTop_df['PitchCall'].isin(['InPlay']) &
        (filteredTop_df['ExitSpeed'] >= 95)).sum() /
        (filteredTop_df['PitchCall'].isin(['InPlay']) &
        filteredTop_df['ExitSpeed'].notna()).sum() if filteredTop_df['PitchCall'].isin(['InPlay']).sum() > 0 else 0
    )
    TopOut_zone = filteredTop_df[~((filteredTop_df['PlateLocSide'].between(-0.951, 0.951)) & (filteredTop_df['PlateLocHeight'].between(1.5, 3.6)))]
    top_o_pitch = len(TopOut_zone)
    top_o_swing = TopOut_zone['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall']).sum()

    stats['Top Zone Chase'] = top_o_swing / top_o_pitch if top_o_pitch > 0 else 0
    stats['Top Zone BABIP'] = (filteredTop_df[filteredTop_df['PlayResult'].isin(['Single', 'Double', 'Triple', 'HomeRun'])]).shape[0] / filteredTop_df[filteredTop_df['PitchCall'] == "InPlay"].shape[0] if filteredTop_df['PitchCall'].isin(['InPlay']).sum() > 0 else 0
    stats['Top Zone SLG'] = (
        filteredTop_df[filteredTop_df['PlayResult'] == "Single"].shape[0] +
        2 * filteredTop_df[filteredTop_df['PlayResult'] == "Double"].shape[0] +
        3 * filteredTop_df[filteredTop_df['PlayResult'] == "Triple"].shape[0] +
        4 * filteredTop_df[filteredTop_df['PlayResult'] == "HomeRun"].shape[0]
    ) / filteredTop_df[filteredTop_df['PitchCall'] == "InPlay"].shape[0] if filteredTop_df['PitchCall'].isin(['InPlay']).sum() > 0 else 0
    stats['Num Bot FB'] = len(filteredBot_df)
    stats['% Bot FB'] = stats['Num Bot FB'] / stats['Total FB']
    stats['Bot Zone VAA'] = filteredBot_df['VertApprAngle'].mean()
    stats['Bot Zone Whiff%'] = filteredBot_df['PitchCall'].isin(['StrikeSwinging']).sum() / filteredBot_df['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall']).sum()
    stats['Bot Zone Hard Hit %'] = (
        (filteredBot_df['PitchCall'].isin(['InPlay']) &
        (filteredBot_df['ExitSpeed'] >= 95)).sum() /
        (filteredBot_df['PitchCall'].isin(['InPlay']) &
        filteredBot_df['ExitSpeed'].notna()).sum()
    )
    BotOut_zone = filteredBot_df[~((filteredBot_df['PlateLocSide'].between(-0.951, 0.951)) & (filteredBot_df['PlateLocHeight'].between(1.5, 3.6)))]
    bot_o_pitch = len(BotOut_zone)
    bot_o_swing = BotOut_zone['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall']).sum()

    stats['Bot Zone Chase'] = bot_o_swing / bot_o_pitch if bot_o_pitch > 0 else 0
    stats['Bot Zone BABIP'] = (filteredBot_df[filteredBot_df['PlayResult'].isin(['Single', 'Double', 'Triple', 'HomeRun'])]).shape[0] / filteredBot_df[filteredBot_df['PitchCall'] == "InPlay"].shape[0] if filteredBot_df['PitchCall'].isin(['InPlay']).sum() > 0 else 0
    stats['Bot Zone SLG'] = (
        filteredBot_df[filteredBot_df['PlayResult'] == "Single"].shape[0] +
        2 * filteredBot_df[filteredBot_df['PlayResult'] == "Double"].shape[0] +
        3 * filteredBot_df[filteredBot_df['PlayResult'] == "Triple"].shape[0] +
        4 * filteredBot_df[filteredBot_df['PlayResult'] == "HomeRun"].shape[0]
    ) / filteredBot_df[filteredBot_df['PitchCall'] == "InPlay"].shape[0] if filteredBot_df['PitchCall'].isin(['InPlay']).sum() > 0 else 0
    stats['Top Location Rank'] = calculate_top_rank(filteredTop_df['VertApprAngle'].mean())
    stats['Bot Location Rank'] = calculate_bot_rank(filteredBot_df['VertApprAngle'].mean())

    return stats


# Loop through each pitcher
for pitcher in returnerNames:
  stats = calculate_stats(topZoneDF, botZoneDF, returnerDF, pitcher)
  if stats is None:
      continue
  new_row = {
    'Name': pitcher,
    'Total FB': stats['Total FB'],
    'Release Height': stats['Release Height'],
    'Num Top FB': stats['Num Top FB'],
    '% Top FB': stats['% Top FB'],
    'Top Zone VAA': stats['Top Zone VAA'],
    'Top Zone Whiff%': stats['Top Zone Whiff%'],
    'Top Zone Chase%': stats['Top Zone Chase'],
    'Top Zone Hard Hit %': stats['Top Zone Hard Hit %'],
    'Top Zone BABIP': stats['Top Zone BABIP'],
    'Top Zone SLG': stats['Top Zone SLG'],
    'Num Bot FB': stats['Num Bot FB'],
    '% Bot FB': stats['% Bot FB'],
    'Bot Zone VAA': stats['Bot Zone VAA'],
    'Bot Zone Whiff%': stats['Bot Zone Whiff%'],
    'Bot Zone Chase%': stats['Bot Zone Chase'],
    'Bot Zone Hard Hit %': stats['Bot Zone Hard Hit %'],
    'Bot Zone BABIP': stats['Bot Zone BABIP'],
    'Bot Zone SLG': stats['Bot Zone SLG'],
    'Top Location Rank': stats['Top Location Rank'],
    'Bot Location Rank': stats['Bot Location Rank'],
    'Whiff %': stats['Whiff %'],
    'Chase %': stats['Chase %'],
    'Hard Hit %': stats['Hard Hit %'],
    'BABIP': stats['BABIP'],
    'SLG': stats['SLG']
  }
  new_row_df = pd.DataFrame([new_row])
  returning_pitcher_VAA = pd.concat([returning_pitcher_VAA, new_row_df], ignore_index=True)

<ipython-input-20-86b5913a5c23>:158: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  returning_pitcher_VAA = pd.concat([returning_pitcher_VAA, new_row_df], ignore_index=True)
<ipython-input-20-86b5913a5c23>:99: RuntimeWarning: invalid value encountered in scalar divide
  stats['Bot Zone Whiff%'] = filteredBot_df['PitchCall'].isin(['StrikeSwinging']).sum() / filteredBot_df['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall']).sum()
<ipython-input-20-86b5913a5c23>:101: RuntimeWarning: invalid value encountered in scalar divide
  (filteredBot_df['PitchCall'].isin(['InPlay']) &


In [21]:
returning_pitcher_VAA

,Name,Total FB,Release Height,Num Top FB,% Top FB,Top Zone VAA,Top Zone Whiff%,Top Zone Chase%,Top Zone Hard Hit %,Top Zone BABIP,...,Bot Zone Hard Hit %,Bot Zone BABIP,Bot Zone SLG,Top Location Rank,Bot Location Rank,Whiff %,Chase %,Hard Hit %,BABIP,SLG
0,"Johnson, Fisher",163,6.109691,22,0.134969,-4.993284,0.555556,0.400000,0.000000,0.000000,...,0.500000,0.222222,0.333333,Below Average High,Below Average Low,0.200000,0.150685,0.307692,0.261905,0.333333
1,"Tostrup, Dylan",50,5.737763,7,0.140000,-4.047932,0.500000,0.250000,1.000000,0.000000,...,NaN,0.000000,0.000000,Above Average High,Above Average Low,0.181818,0.034483,0.500000,0.000000,0.000000
2,"Ebner, Michael",233,6.351534,45,0.193133,-4.710459,0.333333,0.181818,0.375000,0.500000,...,0.500000,0.250000,0.500000,Average High,Average Low,0.349206,0.059406,0.379310,0.225806,0.387097
3,"Matson, Sax",41,5.415380,8,0.195122,-4.092524,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,Above Average High,Great Low,0.250000,0.100000,0.285714,0.250000,1.000000
4,"Di Landri, Tommy",6,5.866225,2,0.333333,-4.559597,0.000000,0.000000,0.000000,1.000000,...,NaN,0.000000,0.000000,Average High,Average Low,0.500000,0.333333,0.000000,0.500000,0.500000
5,"Aoki, Caden",173,5.983415,19,0.109827,-4.848559,0.100000,0.000000,0.428571,0.250000,...,0.750000,0.400000,0.400000,Average High,Average Low,0.089552,0.135135,0.418605,0.354167,0.541667
6,"Blatter, Brock",98,6.174401,17,0.173469,-5.453663,0.800000,0.000000,0.000000,0.000000,...,0.500000,0.500000,0.500000,Poor High,Below Average Low,0.303030,0.090909,0.500000,0.166667,0.166667
7,"Austin, Channing",188,6.166424,35,0.186170,-4.284785,0.272727,0.176471,0.000000,0.000000,...,0.333333,0.333333,0.333333,Above Average High,Above Average Low,0.278689,0.206897,0.260870,0.333333,0.375000
8,"Edwards, Mason",230,5.992037,32,0.139130,-4.452596,0.428571,0.222222,0.800000,0.800000,...,0.428571,0.428571,0.428571,Average High,Above Average Low,0.305556,0.110000,0.432432,0.432432,0.891892
9,"Martinez, Xavier",200,5.401138,32,0.160000,-4.258562,0.529412,0.200000,0.200000,0.000000,...,0.428571,0.428571,0.428571,Above Average High,Above Average Low,0.241935,0.071429,0.388889,0.361111,0.444444


In [22]:
csv = returning_pitcher_VAA.to_csv("/content/drive/Shareddrives/USC Baseball/ML Stuff/VAA/ReturningPitchers2024VAA.csv", index=False)